In [32]:
import sys
path='/home/tomas/Ulmer-Berechnung/alps2qutipplus-april/alps2qutipplus-main/'

sys.path.insert(1, path) 

In [33]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qutip
import scipy.linalg as linalg
import time

from alpsqutip import (build_system, list_models_in_alps_xml,
                       list_geometries_in_alps_xml, graph_from_alps_xml,
                       model_from_alps_xml,
                       restricted_maxent_toolkit as me)

from alpsqutip.operators.states.utils import safe_exp_and_normalize ## function used to safely and robustly map K-states to states

from alpsqutip.operators.states.meanfield.projections import (one_body_from_qutip_operator, 
                                                  project_operator_to_m_body, 
                                                             project_qutip_operator_to_m_body,
                                                             project_to_n_body_operator,
                                                             project_qutip_operator_as_n_body_operator,
                                                             project_product_operator_as_n_body_operator)

from alpsqutip.operators import (
    ScalarOperator,
    LocalOperator,
    OneBodyOperator,
    Operator,
    ProductOperator,
    ScalarOperator,
    SumOperator,
    QutipOperator
)

from alpsqutip.scalarprod import gram_matrix, fetch_covar_scalar_product, orthogonalize_basis
from alpsqutip.operators.states.gibbs import GibbsDensityOperator, GibbsProductDensityOperator

In [34]:
params={}

params['size']=5
params['Jx']=1.; params['Jy'] = .75*params['Jx']; params['Jz']=1.05*params['Jx']

from scipy.optimize import root, fsolve
Ffactor=np.real(max(np.roots(np.poly1d([1, 0, -(params['Jx']*params['Jy']+params['Jx']*params['Jy']+params['Jy']*params['Jz']), 
                           -2*params['Jx']*params['Jy']*params['Jz']]))))
chi_y=fsolve(lambda x,y: x*np.arcsinh(x)-np.sqrt(x**2+1)-y, 1e-1, args=(0))[0]
vLR=4*Ffactor*chi_y

In [35]:
system=build_system(geometry_name= "open chain lattice",model_name="spin", 
                    L=params['size'], J=1)

sites=[s for s in system.sites]
sx_ops=[system.site_operator("Sx", '1[' + str(a) + ']') for a in range(len(system.sites))]
sy_ops=[system.site_operator("Sy", '1[' + str(a) + ']') for a in range(len(system.sites))]
sz_ops=[system.site_operator("Sz", '1[' + str(a) + ']') for a in range(len(system.sites))]

Sz = system.global_operator("Sz")
H = (
    params['Jx'] * sum(sx_ops[i] * sx_ops[i+1] for i in range(len(sites)-1)) +
    params['Jy'] * sum(sy_ops[i] * sy_ops[i+1] for i in range(len(sites)-1)) +
    params['Jz'] * sum(sz_ops[i] * sz_ops[i+1] for i in range(len(sites)-1))
)

idop = [system.site_operator('identity@1[' + str(a) + ']') for a in range(len(system.sites))]
idop = idop[0]*idop[1]*idop[2]*idop[3]#*idop[4]*idop[5]*idop[6]

loading model spin  over graph open chain lattice


In [36]:
HBB0=[idop, 
      system.site_operator('Sz', '1[0]'), 
      system.site_operator('Sz', '1[1]'), 
      system.site_operator('Sz', '1[2]')]


phi0 = np.array([.0, -.25, -.25, 
                 0.75])

K01 = me.k_state_from_phi_basis(phi0, HBB0)
sigma0 = GibbsProductDensityOperator(K01)
phi0[0] = np.log(sigma0.tr())
K0 = me.k_state_from_phi_basis(phi0, HBB0)
sigma0 = GibbsProductDensityOperator(K0)

timespan=np.linspace(.0, 1500.1/vLR,200)
obs_SzA = sum(sz for sz in sz_ops)
[(sigma0 * op).tr() for op in sz_ops] 

[0.0621765008857981, 0.0621765008857981, -0.17917869917539297, 0.0, 0.0]

In [37]:
sigma0

In [38]:
obs_SzA.to_qutip()

Quantum object: dims = [[2, 2, 2, 2, 2], [2, 2, 2, 2, 2]], shape = (32, 32), type = oper, isherm = True
Qobj data =
[[-2.5  0.   0.  ...  0.   0.   0. ]
 [ 0.  -1.5  0.  ...  0.   0.   0. ]
 [ 0.   0.  -1.5 ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ...  1.5  0.   0. ]
 [ 0.   0.   0.  ...  0.   1.5  0. ]
 [ 0.   0.   0.  ...  0.   0.   2.5]]

In [39]:
## exact dynamics
exact_K_at_timet=qutip.mesolve(H=H.to_qutip(), rho0=K0.to_qutip(), tlist=timespan)
exact_rho_at_timet=[safe_exp_and_normalize(-K_at_timeti)[0] for K_at_timeti in exact_K_at_timet.states]

evs_obs_ex=[qutip.expect(rho_at_timeti, obs_SzA.to_qutip()) for rho_at_timeti in exact_rho_at_timet]

In [40]:
simulations[2] = {
    "parms":{
    "chosen_depth": 5,
    "m0": 3,
    "eps": 1e-3,
    },
  "saved_cut_times_index_ell":[0],
   "no_acts_ell" : [0],
    "local_bound_error_ell":[],
    "spectral_norm_Hij_tensor_ell":[],
    "instantaneous_w_errors":[],
} 

current_simulation = simulations[2]
current_simulation_parms = current_simulation["parms"]
chosen_depth = current_simulation_parms["chosen_depth"]
eps_tol = current_simulation_parms["eps"]
m0 = current_simulation_parms["m0"]

In [ ]:
# phi0_proj_act_MF = []; local_bound_error_ell=[]
local_bound_error_ell = []
simulations[1]["evs"] = [(sigma0.to_qutip()* obs_SzA.to_qutip()).tr()]

sp_local=fetch_covar_scalar_product(sigma=sigma0); local_t_value=.0

HBB_ell_act = me.build_hierarchical_basis(generator=H.to_qutip_operator(),
                                          seed_op=K0.to_qutip_operator(),
                                          deep=chosen_depth,
                                          to_qutip_operator=True)

HBB_ell_act = [op.to_qutip_operator() for op in HBB_ell_act]

orth_basis_act = orthogonalize_basis(basis=HBB_ell_act,
                                        sp = sp_local)

Hij_tensor_act, w_errors = me.fn_hij_tensor_with_errors(generator=H.to_qutip_operator(), 
                                                        basis=orth_basis_act,
                                                        sp=sp_local)

phi0_proj_act = [sp_local(K0, op) for op in orth_basis_act]
sigma_at_timet = [GibbsProductDensityOperator(K0)]  
sigma_act = sigma_at_timet[0]

for t in timespan[1:]:
    phi_local=np.real(linalg.expm(Hij_tensor_act * (t - local_t_value)) @ phi0_proj_act)  
    
    print(f"A. phi-local, {time.time() - start:.2f}s");   start=time.time();
    
    K_local=me.k_state_from_phi_basis(phi=phi_local,
                                        basis=orth_basis_act) ### i put the minus in order to get a plusK
    
    print(f"B. K-local, {time.time() - start:.2f}s");   start=time.time();
        
    simulations[1]["evs"].append((GibbsDensityOperator(K_local)*obs_SzA).tr()) ### the minus is here
    
    print(f"C. obs-calc, {time.time() - start:.2f}s");   start=time.time();
    
    local_bound_error_ell.append(me.m_th_partial_sum(phi=phi_local, m=2)
                                 / me.m_th_partial_sum(phi=phi_local, m=0))
    
    if abs(local_bound_error_ell[-1]) >= eps_tol:
        
        local_t_value = t ; start = time.time()
        
        print("a. start t=", list(timespan).index(t))
        if True:
            K_act=project_to_n_body_operator(operator=K_local.as_sum_of_products(),
                                         sigma=sigma_act, 
                                         nmax=1)
            sigma_act=GibbsProductDensityOperator(K_act)
            K_act_at_timet.append(K_act)        
        
        end = time.time(); print(f"b. Kact-m1proj, {end - start:.3f}s")
        
        start = time.time()
        
        sp_local = fetch_covar_scalar_product(sigma=sigma0)  
        HBB_ell_act = me.build_hierarchical_basis(generator=H.to_qutip_operator(),
                                                            seed_op=K_local.to_qutip_operator(), 
                                                            deep=chosen_depth,
                                                            to_qutip_operator=False)[1:] + [K_act.to_qutip_operator()]
        
        HBB_ell_act = [op.simplify() for op in HBB_ell_act]
        
        end = time.time(); print(f"c. HBact constr., {end - start:.6f}s")
        
        start = time.time()
        
        HBB_ell_act= ([op for op in HBB_ell_act[0:0]] + [project_qutip_operator_as_n_body_operator
                                              (operator = op.as_sum_of_products(), 
                                               nmax=m0,
                                               sigma=sigma_act).to_qutip_operator()
                      for op in HBB_ell_act][0:])
        
        end = time.time(); print(f"d. HBact proj, {end - start:.6f}s")
        orth_basis_act = orthogonalize_basis(basis=[K_local.to_qutip_operator()]+HBB_ell_act,
                                             sp=sp_local)  
        
        start = time.time()
        
        Hij_tensor_act, w_errors = me.fn_hij_tensor_with_errors(generator=H.to_qutip_operator(), 
                                                                basis=orth_basis_act,
                                                                sp=sp_local)
        
        end = time.time(); print(f"e. Hijact constr, {end - start:.6f}s")
        print(linalg.norm(Hij_tensor_act))
        
        phi0_proj_act = np.array([np.real(sp_local(K_local, op)) for op in orth_basis_act] ).round(10)
        new_K_local=sum(phi0a * opa for phi0a, opa in zip(phi0_proj_act, orth_basis_act))### same here
        print("Fita: Check", linalg.norm((new_K_local - K_local).to_qutip()))
    else:
        # If error is below the threshold, retain the current basis and 
        pass

A. phi-local, 42.07s
B. K-local, 0.00s
C. obs-calc, 0.14s
A. phi-local, 0.01s
B. K-local, 0.00s
C. obs-calc, 0.15s
a. start t= 2
b. Kact-m1proj, 1.695s
c. HBact constr., 0.057658s
d. HBact proj, 9.527515s
e. Hijact constr, 0.191817s
2.7598453737504425
Fita: Check 1.0531895121828302e-10
A. phi-local, 0.22s
B. K-local, 0.00s
C. obs-calc, 0.12s
a. start t= 3
b. Kact-m1proj, 1.499s
c. HBact constr., 0.065800s
d. HBact proj, 12.361561s
e. Hijact constr, 1.072190s
2.7898446741590774
Fita: Check 8.068640883645801e-11
A. phi-local, 1.16s
B. K-local, 0.00s
C. obs-calc, 0.16s
a. start t= 4
b. Kact-m1proj, 1.727s
c. HBact constr., 0.009523s
d. HBact proj, 13.289954s
e. Hijact constr, 0.248865s
2.7429190860300445
Fita: Check 2.340221218541039e-10
A. phi-local, 0.28s
B. K-local, 0.00s
C. obs-calc, 0.12s
A. phi-local, 0.00s
B. K-local, 0.00s
C. obs-calc, 0.15s
a. start t= 6
b. Kact-m1proj, 1.612s
c. HBact constr., 0.011081s
d. HBact proj, 13.613945s
e. Hijact constr, 0.200246s
3.310562859835667
Fita

In [ ]:
simulations[2] = {
    "parms":{
    "chosen_depth": 5,
    "m0": 3,
    "eps": 1e-3,
    },
  "saved_cut_times_index_ell":[0],
   "no_acts_ell" : [0],
    "local_bound_error_ell":[],
    "spectral_norm_Hij_tensor_ell":[],
    "instantaneous_w_errors":[],
} 

current_simulation = simulations[2]
current_simulation_parms = current_simulation["parms"]
chosen_depth = current_simulation_parms["chosen_depth"]
eps_tol = current_simulation_parms["eps"]
m0 = current_simulation_parms["m0"]

In [ ]:
# phi0_proj_act_MF = []; local_bound_error_ell=[]
local_bound_error_ell = []
simulations[2]["evs"] = [(sigma0.to_qutip()* obs_SzA.to_qutip()).tr()]

sp_local=fetch_covar_scalar_product(sigma=sigma0); local_t_value=.0

HBB_ell_act = me.build_hierarchical_basis(generator=H.to_qutip_operator(),
                                          seed_op=K0.to_qutip_operator(),
                                          deep=chosen_depth,
                                          to_qutip_operator=True)

HBB_ell_act = [op.simplify().to_qutip_operator() for op in HBB_ell_act]

orth_basis_act = orthogonalize_basis(basis=HBB_ell_act,
                                        sp = sp_local)

Hij_tensor_act, w_errors = me.fn_hij_tensor_with_errors(generator=H.to_qutip_operator(), 
                                                        basis=orth_basis_act,
                                                        sp=sp_local)

phi0_proj_act = [sp_local(K0, op) for op in orth_basis_act]
sigma_act = sigma_at_timet[0]

for t in timespan[1:]:
    
    start = time.time()
    phi_local=np.real(linalg.expm(Hij_tensor_act * (t - local_t_value)) @ phi0_proj_act)  
    
    print(f"A. phi-local, {time.time() - start:.2f}s");   start=time.time();
    
    K_local=me.k_state_from_phi_basis(phi=phi_local,
                                        basis=orth_basis_act) ### i put the minus in order to get a plusK
    
    
    print(f"B. K-local, {time.time() - start:.2f}s");   start=time.time();
    
    simulations[2]["evs"].append((GibbsDensityOperator(K_local.to_qutip_operator())
                                  *obs_SzA.to_qutip_operator()).tr()) ### the minus is here
    
    print(f"C. obs-calc, {time.time() - start:.2f}s");   start=time.time();
        
    local_bound_error_ell.append(me.m_th_partial_sum(phi=phi_local, m=2)
                                 / me.m_th_partial_sum(phi=phi_local, m=0))
    
    if abs(local_bound_error_ell[-1]) >= eps_tol:
        
        local_t_value = t ; start = time.time()
        
        print("___a. start act t=", list(timespan).index(t))
        
        start = time.time()
        
        K_proj_m0=project_to_n_body_operator(operator=K_local.as_sum_of_products(),
                                         sigma=sigma_act, 
                                         nmax=m0).simplify()
        
        end = time.time(); print(f"___b. Kact-m0proj, {end - start:.2f}s")
        
        if True:
            K_act=project_to_n_body_operator(operator=K_proj_m0.as_sum_of_products(),
                                         sigma=sigma_act, 
                                         nmax=1).simplify()
            sigma_act=GibbsProductDensityOperator(K_act)
            K_act_at_timet.append(K_act)        
            
            
        end = time.time(); print(f"___c. Kact-m1proj, {end - start:.2f}s")
        
        sp_local = fetch_covar_scalar_product(sigma=sigma0)  
        
        HBB_ell_act = me.build_hierarchical_basis(generator=H.to_qutip_operator(),
                                                            seed_op=K_proj_m0.to_qutip_operator(), 
                                                            deep=chosen_depth,
                                                            to_qutip_operator=False)[1:]
        
        HBB_ell_act = [op.simplify() for op in HBB_ell_act]
        
        end = time.time(); print(f"___d. HBact proj, {end - start:.2f}s")
        orth_basis_act = orthogonalize_basis(basis=[K_local.to_qutip_operator()]+HBB_ell_act,
                                             sp=sp_local)  
        
        start = time.time()
        
        Hij_tensor_act, w_errors = me.fn_hij_tensor_with_errors(generator=H.to_qutip_operator(), 
                                                                basis=orth_basis_act,
                                                                sp=sp_local)
        
        end = time.time(); print(f"___e. Hijact constr, {end - start:.2f}s")
        
        phi0_proj_act = np.array([np.real(sp_local(K_local, op)) for op in orth_basis_act] ).round(10)
        new_K_local=sum(phi0a * opa for phi0a, opa in zip(phi0_proj_act, orth_basis_act))### same here
        print("Fita: Check", linalg.norm((new_K_local - K_local).to_qutip()))
    else:
        # If error is below the threshold, retain the current basis and 
        pass

In [ ]:
import matplotlib.pyplot as plt

# Création de la figure et des axes
fig, ax = plt.subplots(figsize=(6, 6))

n = 1

# Tracé de la courbe exacte
ax.plot(timespan*vLR, evs_obs_ex, label='Exact', color="blue", linestyle='dashdot', linewidth=1)

# Récupération des paramètres pour formater dynamiquement le label
l = simulations[2]["parms"]["chosen_depth"]
m0 = simulations[2]["parms"]["m0"]
eps = simulations[2]["parms"]["eps"]

label_sim1 = fr'Max-Ent $(\ell, m_0, \epsilon) = ({l}, {m0}, {eps})$'


# Tracé de la simulation 1
ax.plot(timespan*vLR, simulations[1]["evs"], 
        label=label_sim1, 
        color='red', linestyle='--', marker='D', linewidth=1)

ax.plot(timespan*vLR, simulations[2]["evs"], 
        label=label_sim1, 
        color='blue', linestyle='--', marker='D', linewidth=1)

# Bordures uniquement à gauche et en bas (style minimaliste)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Personnalisation des ticks
ax.tick_params(direction='out', length=6, width=2, which='major', grid_alpha=0.6)
ax.grid(True, which='major', linestyle='--', linewidth=0.75, alpha=0.7)

ax.legend(frameon=False, loc='upper right')

# Axe secondaire
ax1 = ax.twiny()
ax1.set_xlim(timespan[0]/vLR, timespan[-1]/vLR)

# Étiquettes des axes
ax.set_xlabel(r'$t v_{\rm LR} \, [L]$')
ax1.set_xlabel(r'$t \, [{\rm arb. units}]$')

# Titre formaté avec LaTeX
plt.title(r'$L={}$, $J_x={}$, $v_{{\rm LR}}={}$'.format(
    params['size'], 
    round(params['Jx'], 3), 
    round(vLR, 3)
))

# Ajustement final
plt.tight_layout()


In [193]:
tgt=QutipOperator(exact_K_at_timet.states[20])


In [194]:
linalg.norm((tgt - project_qutip_operator_as_n_body_operator(QutipOperator(exact_K_at_timet.states[20]),
                                             3).to_qutip_operator()).to_qutip())/linalg.norm(tgt.to_qutip())

0.5716902617759407

In [195]:
(exact_rho_at_timet[20]*obs_SzA.to_qutip()).tr()

-0.03089911784316446

In [196]:
simulations[1]["evs"][20]

(-0.001776569602615019+2.0316608732545758e-10j)

In [192]:
(GibbsDensityOperator(project_qutip_operator_as_n_body_operator(QutipOperator(exact_K_at_timet.states[7]),
                                             3)).to_qutip()*obs_SzA.to_qutip()).tr()

-0.056350565346876116

In [96]:
phitgt = (qutip.Qobj(Hij_tensor_act)*(24-timespan[-1])).expm()*phi0_proj_act

(GibbsDensityOperator(phitgt@orth_basis_act)*obs_SzA).tr()

-0.0007843571168305308

In [98]:
((-me.k_state_from_phi_basis(phitgt, orth_basis_act)).expm()*obs_SzA).tr()

-0.013845743677530442

In [58]:
linalg.norm(K0.to_qutip())

2.1213203435596424

In [60]:
linalg.norm(K_local.to_qutip())

2.427543480535756

In [49]:
qutip.Qobj(Hij_tensor_act)

Quantum object: dims = [[7], [7]], shape = (7, 7), type = oper, isherm = False
Qobj data =
[[ 5.77550528e-02 -2.30574297e+00 -1.04750853e-01 -1.80685880e-01
  -5.23897026e-03 -9.12087778e-02 -5.72135174e-02]
 [ 2.28693526e+00 -5.83729288e-02  1.48003244e-01  1.79648669e-01
  -6.36887826e-02  1.14000763e-01  5.28466512e-02]
 [-4.82167621e-12 -3.15040401e-04 -4.56580884e-03 -3.63053149e-01
   4.52434668e-03  2.19919985e-01 -2.64092674e-01]
 [-1.03844862e-12  2.05330976e-03  4.18265367e-01 -2.05701091e-02
  -4.57784461e-01 -1.46840985e-01  2.75730150e-01]
 [-5.91628591e-12 -7.19628855e-04  5.12746774e-02  4.21831065e-01
   2.40333288e-02 -9.11922801e-02 -1.60676984e-01]
 [-5.78605501e-12  6.08011369e-03 -2.12311291e-01  1.32025757e-01
   3.01336416e-01 -8.46557818e-02  3.47022456e-02]
 [ 1.64308793e-12  6.94119925e-03  2.74528129e-01 -2.85523491e-01
   2.14211045e-01 -1.39365520e-02  8.86307019e-03]]

In [50]:
phi0_proj_act

array([ 0.62191134, -0.        ,  0.        ,  0.        ,  0.        ,
       -0.        ,  0.        ])

In [34]:
sp_local(K_local - HBB_ell_act[1], K_local - HBB_ell_act[1])

3.4120595220378105

In [151]:
simulations={}
simulations[1] = {
    "parms":{
    "chosen_depth": 4,
    "m0": 2,
    "eps": 1e-3,
    },
  "saved_cut_times_index_ell":[0],
   "no_acts_ell" : [0],
    "local_bound_error_ell":[],
    "spectral_norm_Hij_tensor_ell":[],
    "instantaneous_w_errors":[],
} 

current_simulation = simulations[1]
current_simulation_parms = current_simulation["parms"]
chosen_depth = current_simulation_parms["chosen_depth"]
eps_tol = current_simulation_parms["eps"]
m0 = current_simulation_parms["m0"]

In [152]:
phi0_proj_act_MF = []; local_bound_error_ell=[]

simulations[1]["evs"] = [(sigma0.to_qutip()* obs_SzA.to_qutip()).tr()]

sp_local=fetch_covar_scalar_product(sigma=sigma0); local_t_value=.0

HBB_ell_act = me.build_hierarchical_basis(generator=H,
                                          seed_op=K0,
                                          deep=chosen_depth,
                                          to_qutip_operator=False)

HBB_ell_act = [op for op in HBB_ell_act]

orth_basis_act = orthogonalize_basis(basis=HBB_ell_act,
                                        sp = sp_local)

Hij_tensor_act, w_errors = me.fn_hij_tensor_with_errors(generator=H, 
                                                        basis=orth_basis_act,
                                                        sp=sp_local)

phi0_proj_act = [sp_local(K0, op) for op in orth_basis_act]
phi_at_timet = [phi0_proj_act]  
K_at_timet = [K0]; K_act_at_timet=[K0]  
sigma_at_timet = [GibbsProductDensityOperator(K0)]  
sigma_act = sigma_at_timet[0]

for t in timespan[1:]:
    phi_local=np.real(linalg.expm(Hij_tensor_act * (t - local_t_value)) @ phi0_proj_act)  
    K_local=me.k_state_from_phi_basis(phi=phi_local,
                                        basis=orth_basis_act) ### i put the minus in order to get a plusK
        
    simulations[1]["evs"].append((GibbsDensityOperator(K_local)*obs_SzA).tr()) ### the minus is here
    
    phi_at_timet.append(phi_local); K_at_timet.append(K_local)
    
    local_bound_error_ell.append(me.m_th_partial_sum(phi=phi_local, m=2)
                                 / me.m_th_partial_sum(phi=phi_local, m=0))
    
    if abs(local_bound_error_ell[-1]) >= eps_tol:
        
        local_t_value = t ; start = time.time()
        
        print("a. start t=", list(timespan).index(t))
        
        K_proj_m0 = project_to_n_body_operator(
                        operator = K_local.to_qutip_operator(),
                        sigma = None,
                        nmax=m0
        )
        
        end = time.time(); print(f"b. PiKlocal, {end - start:.3f}")
        start = time.time()
        
        K_act = project_to_n_body_operator(
                                            operator=K_proj_m0,
                                            sigma=None, 
                                            nmax=1
        )
        
        sigma_act=GibbsProductDensityOperator(K_act)
        K_act_at_timet.append(K_act)        
        
        end = time.time(); print(f"b. Kact-m1proj, {end - start:.3f}")
        start = time.time()
        
        sp_local = fetch_covar_scalar_product(sigma=sigma_act)  
        
        HBB_ell_act = [K_local] + me.build_hierarchical_basis(
                                        generator=H.to_qutip_operator(),
                                        seed_op = K_proj_m0.to_qutip_operator(),
                                        deep = chosen_depth-1, 
                                        to_qutip_operator=False
        )
        
        end = time.time(); print(f"d. HBact, {end - start:.3f}")
        start = time.time()
        
        orth_basis_act = orthogonalize_basis(basis=HBB_ell_act,
                                                sp = sp_local)
        
        end = time.time(); print(f"e. orth-basis-act, {end - start:.3f}")
        start = time.time()
        
        Hij_tensor_act, errors = me.fn_hij_tensor_with_errors(basis=orth_basis_act,
                                                              sp=sp_local, 
                                                              generator=H.to_qutip_operator())
        end = time.time(); print(f"f. Hij-tensor-act, {end - start:.3f}")
        
        phi0_proj_act = np.array([sp_local(K_local, op) for op in orth_basis_act]).round(9)
        new_K_local=sum(phi0a * opa for phi0a, opa in zip(phi0_proj_act, orth_basis_act))
        print("Fita: Check", linalg.norm((new_K_local - K_local).to_qutip()))
        
    else:
        # If error is below the threshold, retain the current basis and 
        K_act_at_timet.append(K_local)

a. start t= 1
b. PiKlocal, 0.356
b. Kact-m1proj, 0.023
d. HBact, 0.009
e. orth-basis-act, 3.146
f. Hij-tensor-act, 0.312
Fita: Check 0.0
a. start t= 2
b. PiKlocal, 0.403
b. Kact-m1proj, 0.027
d. HBact, 0.009
e. orth-basis-act, 3.551
f. Hij-tensor-act, 0.247
Fita: Check 0.0
a. start t= 3
b. PiKlocal, 0.301
b. Kact-m1proj, 0.016
d. HBact, 0.006
e. orth-basis-act, 2.600
f. Hij-tensor-act, 0.267
Fita: Check 0.0
a. start t= 4
b. PiKlocal, 0.288
b. Kact-m1proj, 0.017
d. HBact, 0.006
e. orth-basis-act, 3.862
f. Hij-tensor-act, 0.293
Fita: Check 0.0
a. start t= 5
b. PiKlocal, 0.310
b. Kact-m1proj, 0.016
d. HBact, 0.006
e. orth-basis-act, 4.696
f. Hij-tensor-act, 0.290
Fita: Check 0.0
a. start t= 6
b. PiKlocal, 0.307
b. Kact-m1proj, 0.016
d. HBact, 0.006
e. orth-basis-act, 4.741
f. Hij-tensor-act, 0.293
Fita: Check 0.0
a. start t= 7
b. PiKlocal, 0.317
b. Kact-m1proj, 0.016
d. HBact, 0.006
e. orth-basis-act, 4.686
f. Hij-tensor-act, 0.291
Fita: Check 0.0
a. start t= 8
b. PiKlocal, 0.356
b. Kact-

KeyboardInterrupt: 